# Gráficas de visibilidad en trenes de disparos en los ganglios basales

### Marco Arieli Herrera-Valdez$^1$, Alejandra Verónica Cáceres$^2$, José Bargas$^2$
#### $^1$Facultad de Ciencias, $^2$Instituto de Fisiología Celular, División de Neurociencias, 
#### Universidad Nacional Autónoma de México


Cálculo de gráficas de visibilidad en trenes de disparos registrados con patch clamp.

In [6]:
import scipy as sc
sc.test("all")
import pylab as gr
import io
import networkx as nx

Running unit tests for scipy
NumPy version 1.10.4
NumPy relaxed strides checking option: False
NumPy is installed in /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/numpy
SciPy version 0.17.0
SciPy is installed in /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy
Python version 3.5.1 (v3.5.1:37a07cee5969, Dec  5 2015, 21:12:44) [GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]
nose version 1.3.7


.
----------------------------------------------------------------------
Ran 1 test in 2.545s

OK


Funciones de soporte

In [7]:
def getFileList(dataDir, prefix, suffix):
    files=list()
    for f in os.listdir(dataDir):
        a=os.path.isfile(os.path.join(dataDir,f))
        b=string.find(f,prefix)>-1
        c=string.find(f,suffix)>0
        #print(f,a,b,c)
        if (a*b*c):
            files.append(f)
    nFiles = len(files)
    print("Found %d files with the indicated string"%nFiles)
    print(files)
    return files

def slopeAB(A,B):
    """
    slopeAB calculates the slope between two points A and B
    Example:
    sAB=slopeAB(a,b)
    """
    mAB = (B[1]-A[1]) / (B[0]-A[0])
    return mAB


def calcFwdSlopes(x,y): 
    """
    calcFwdSlopes calculates the forward-slopes between pairs of points from graph {(x,y)}
    Example:
    s=calcFwdSlopes(x,y)
    """
    nY=len(y)
    s=list()
    for m in sc.arange(nY-1):
        A=[x[m], y[m]]
        s.append(list())
        for n in sc.arange(m+1,nY):
            B=[x[n], y[n]]
            s[m].append(slopeAB(A,B))
    return s 

def fwdVisibilityInds(x,y):
    """
    fwdVisibilityInds obtains the indices of the forward-visible neighbours within a graph {(x,y)}
    neighs=fwdVisibilityInds(x,y)
    """
    nodes=list()
    neighs=list()
    nY=len(y)
    s=calcFwdSlopes(x,y)
    #nodes = sc.arange(nY)
    for m in sc.arange(nY-2):
        a=[x[m], y[m]]
        b=[x[m+1], y[m+1]]
        sAB=slopeAB(a,b)
        neighs.append(list())
        neighs[m].append(m+1)
        for n in sc.arange(m+2,len(s[m])):
            if sAB < s[m][n]:
                sAB=s[m][n]
                neighs[m].append(n)
    return neighs

In [8]:
dataDir= "./trenesBursts/aleSpikeTimes/"
fName = "CntSt1.mat"
pigas = sc.squeeze(io.loadmat(dataDir + fName)["savevar"][0][0])

AttributeError: module 'io' has no attribute 'loadmat'

In [ ]:
isi = sc.zeros(len(pigas))
isi[1:]= pigas[1:]-pigas[:-1]
ifr= 1/isi
series= ifr[0:len(ifr)]
print("Calculating the visibility graph of the time series (%d points)"%len(series))
startTime= time.time()
g = visibility_graph( series)
totTime= time.time() - startTime
print("Took %d seconds to calculate the graph"%totTime)
gr.figure()
nx.draw(g,node_color="b",node_size=10)
gr.show()
